# 네이버 뉴스 API 연결해서 뉴스 스크래핑
* 뉴스 홈페이지마다 각기 다른 태그로 본문을 구성하고 있음
* 본문의 내용을 가져오기 위해서 모든 뉴스 홈페이지의 크롤링 코드를 작성하는 것은 불가능
* 웹 스크래핑 & 기사 html 코드 분석
    * webbaseloader 라이브러리를 이용해서 글자가 300자 이상인 것들은 본문으로 생각(광고 등 다른 text가 들어오는 것을 막음)
    * 몇몇 기사에서 담화 형식의 기사를 내보내어 한 문단에 300자가 안되는 경우 발생
    * 담화 형식은 광고의 제목과 비슷한 글자수로 구성되어 있어, 결국 본문이 쓰여있는 div를 찾는 것이 중요
    * 본문이 담겨있는 특정 div에는 br태그나 p태그가 많은 것을 확인하여 이러한 태그가 가장 많은 하위 div 태그를 찾도록 코딩

[참고 자료]
* https://developers.naver.com/docs/serviceapi/search/blog/blog.md#%EA%B2%80%EC%83%89-api-%EB%B8%94%EB%A1%9C%EA%B7%B8-%EA%B2%80%EC%83%89-%EA%B5%AC%ED%98%84-%EC%98%88%EC%A0%9C
* https://contents.premium.naver.com/chatgpt/buff/contents/231204223117640hg


In [136]:
import urllib.request
import pandas as pd
import json
import re
import html

# 네이버 API의 클라이언트 ID와 시크릿 키 설정
MY_CLIENT_ID = "hZm0ny7bv69wAHcdYffq"
MY_CLIENT_SECRET = "pIyAPPtzD2"
CLIENT_ID = MY_CLIENT_ID  # 실제 클라이언트 ID를 입력하세요
CLIENT_SECRET = MY_CLIENT_SECRET  # 실제 클라이언트 시크릿을 입력하세요

# 검색할 키워드 목록
SEARCH_QUERIES = ["주식", "채권", "금융"]

# API 호출 시 설정
DISPLAY_COUNT = 100  # 한 페이지에 표시할 뉴스 항목 수
START_INDEX = 1  # 검색 결과의 시작 인덱스
END_INDEX = 300  # 검색 결과의 종료 인덱스
SORT_ORDER = "sim"  # 정렬 방식: "sim" (유사도) 또는 "date" (날짜)

# 뉴스 데이터를 저장할 데이터프레임 초기화
news_df = pd.DataFrame(columns=["Category", "Title", "Original Link", "Description", "Publication Date"])

def fetch_news(query):
    """
    주어진 검색어에 대해 뉴스 데이터를 가져와서 데이터프레임에 추가합니다.
    
    :param query: 검색어
    """
    global news_df
    encoded_query = urllib.parse.quote(query)  # 검색어를 URL 인코딩
    current_index = len(news_df)  # 현재 데이터프레임의 인덱스
    
    for start_index in range(START_INDEX, END_INDEX, DISPLAY_COUNT):
        url = f"https://openapi.naver.com/v1/search/news?query={encoded_query}&display={DISPLAY_COUNT}&start={start_index}&sort={SORT_ORDER}"
        
        try:
            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", CLIENT_ID)
            request.add_header("X-Naver-Client-Secret", CLIENT_SECRET)
            response = urllib.request.urlopen(request)
            response_code = response.getcode()
            
            if response_code == 200:
                response_body = response.read()
                response_dict = json.loads(response_body.decode('utf-8'))
                items = response_dict['items']
                
                for item in items:
                    # HTML 태그 제거
                    clean_title = re.sub(re.compile('<.*?>'), '', item['title'])
                    clean_description = re.sub(re.compile('<.*?>'), '', item['description'])
                    clean_pub_date = re.sub(re.compile('<.*?>'), '', item['pubDate'])
                    
                    # 데이터프레임에 뉴스 항목 추가
                    news_df.loc[current_index] = [
                        query,  # 카테고리 추가
                        html.unescape(clean_title),  # HTML 엔티티 제거
                        html.unescape(item['originallink']),  # HTML 엔티티 제거
                        html.unescape(clean_description),
                        clean_pub_date
                    ]
                    current_index += 1
            else:
                print(f"Error Code: {response_code}")
                error_details = response.read().decode('utf-8')
                print(f"Error Details: {error_details}")

        except urllib.error.HTTPError as e:
            print(f"HTTPError: {e.code} - {e.reason}")
            error_details = e.read().decode('utf-8')
            print(f"Error Details: {error_details}")

# 각 검색어에 대해 뉴스 데이터 가져오기
for query in SEARCH_QUERIES:
    fetch_news(query)

# 데이터프레임을 CSV 파일로 저장
news_df.to_csv('news_data.csv', index=False)
print("CSV 파일이 'news_data.csv'로 저장되었습니다.")

# 데이터프레임 확인
news_df

CSV 파일이 'news_data.csv'로 저장되었습니다.


,Category,Title,Original Link,Description,Publication Date
0,주식,"국민연금, 비트코인 최다 보유 기업 주식 매입…460억 원 규모",https://news.tvchosun.com/site/data/html_dir/2...,마이크로스트레티지 주식 약 3400만 달러(460억 원) 상당을 매입했다고 보도했다...,"Sat, 17 Aug 2024 11:21:00 +0900"
1,주식,국민연금이 460억 샀다...비트코인 대신 주식,http://www.wowtv.co.kr/NewsCenter/News/Read?ar...,마이크로스트래티지 주식을 매입했다고 보도했다. 마이크로스트래티지는 비트코인을 가장 ...,"Sat, 17 Aug 2024 07:03:00 +0900"
2,주식,"국민연금, 가상화폐 간접 투자… 美 비트코인 최다 보유 기업 주식 매입",https://biz.chosun.com/international/internati...,마이크로스트래티지 주식을 매입했다고 보도했다. 마이크로스트래티지는 비트코인을 가장 ...,"Sat, 17 Aug 2024 09:40:00 +0900"
3,주식,"국민연금, 코인 간접 투자...비트코인 최다 보유 美기업 주식 매입",http://www.fnnews.com/news/202408171033168529,마이크로스트래티지 주식을 매입했다고 보도했다. 마이크로스트래티지는 가장 많은 비트코...,"Sat, 17 Aug 2024 10:44:00 +0900"
4,주식,"국민연금, 비트코인 최다 보유 美마이크로스트래티지 주식 460억 원 규모 매입",https://www.nocutnews.co.kr/news/6197046?utm_s...,마이크로스트래티지 주식을 매입했다고 보도했다. 마이크로스트래티지는 가장 많은 비트코...,"Sat, 17 Aug 2024 09:05:00 +0900"
...,...,...,...,...,...
895,금융,"토스뱅크, 고령층 대상 ‘디지털 금융 교육’ 실시",https://hobbyen-news.com/news/view/10655728043...,토스뱅크가 비대면 금융 거래 확대로 인해 금융 서비스 이용에 어려움을 겪는 금융 취...,"Fri, 16 Aug 2024 09:30:00 +0900"
896,금융,"토스뱅크, 금융취약계층 보호 위한 고령층 디지털 금융 교육",https://www.pointdaily.co.kr/news/articleView....,토스뱅크가 최근 서울 금천구에 위치한 서울시평생교육진흥원 서울시민대학 '모두의학교 ...,"Fri, 16 Aug 2024 09:58:00 +0900"
897,금융,"하나금융그룹, 지난해 ESG 활동으로 5조 4072억원 사회적 가치 창출",https://www.newsquest.co.kr/news/articleView.h...,하나금융그룹이 지난 한 해 동안 환경‧사회‧지배구조 및 경제 간접 기여 등 각종 E...,"Fri, 16 Aug 2024 15:54:00 +0900"
898,금융,은행 AI 혁신 막던 '망 분리' 폐지,https://www.hankyung.com/article/2024081314891,앞으로 금융회사도 내부 PC에서 챗GPT 등 외부와 연결된 인공지능(AI) 서비스를...,"Tue, 13 Aug 2024 17:58:00 +0900"


# 뉴스 링크에 접근해서 원문 스크래핑 (실패)
p 태그인건 다 갖고오려고 했는데, 뭔가 뉴스 링크 들어가면 뉴스가 아니라 다른게 뜨는 듯...

In [11]:
import requests
from bs4 import BeautifulSoup

# 뉴스 내용 추출 함수
def extract_news_content(url):
    """
    주어진 URL에서 뉴스 내용을 추출합니다.
    :param url: 뉴스 URL
    :return: 추출된 뉴스 내용
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([p.text for p in paragraphs])
    return content[:1000]  # 첫 1000자만 반환

In [12]:
import pandas as pd

# CSV 파일에서 데이터프레임으로 읽어오기
news_df = pd.read_csv('news_data.csv')

# 데이터프레임 확인
news_df['Original Link'][0]

'https://www.hankyung.com/article/2024081288721'

In [13]:
# 뉴스 추출해보기
extract_news_content(news_df['Original Link'][0])

'한국경제 회원이 되어 보세요 지금 바로 한국경제 회원으로 가입하시고, 독점 혜택을 누려보세요 \n                        이미 회원이시면 로그인을 클릭해 주세요\n                      계정관리 마이뉴스 기자 구독 관리 마이증권 내 포트폴리오 관리 ⓒ 한경닷컴, 무단전재 및 재배포 금지 \n\n                                                                요양원 규제는 그대로…반쪽짜리 완화 비판도\n                                                            \n \n                                                            앞으로 보험회사가 방문 요양 서비스 시장에 직접 진출할 수 있게 된다. 지금까지는 자회사를 통해 제한적으로만 허용됐다. 다만 요양원 등 요양시설 사업은 여전히 진입 규제에 막혀 있어 일각에서는 ‘반쪽짜리&...\n                                                         \n\n                                                                "여행상품 구매자도 책임 있다"…커지는 티메프 환불 갈등\n                                                            \n \n                                                            티몬·위메프(티메프)의 여행상품과 상품권 환불 책임을 둘러싼 갈등이 커지고 있다. 환불금을 놓고 카드사와 전자지급결제대행(PG)사, 여행사 간 갈등이 불거진 가운데 소비자도 계약 상대방으로서 일정 부분 ...\n                                                         \n\n                    

# LangChain WebBaseLoader 활용 (반만 됨...-앵커와의 담화문 기사 같은 경우에는(0번 뉴스) 문장별로 개행이 되어있고 글이 짧아서 아무것도 안들어감)
한줄기의 희망!!! 전체 문서를 다 가져와줌 <br>
이걸 이용해서 개행문자 기준으로 나눴을 때, 글자 길이가 길면 본문으로 간주 <br>
글자 길이 : 300자 <br>
(100자는 옆에 뜨는 광고도 다 가져오고, 너무 크면 가끔 문단을 나눈다고 p태그 여러개 적용해서 뉴스 기사 쓴 것들이 있어서 원문 소실됨)

In [137]:
# 요청실패와 피싱실패 예외 처리
# 태그 카운트 최적화
# 숨겨진 콘텐츠 무시

from bs4 import BeautifulSoup
import requests
import certifi
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def extract_news_content(url):
    # 체크할 태그 목록 설정
    tags_to_check = ['br', 'p']
    
    # User-Agent 헤더 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }
    
    try:
        # URL에서 HTML 콘텐츠 가져오기
        response = requests.get(url, headers=headers, verify=certifi.where())
        response.raise_for_status()  # HTTP 오류 확인
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
    except requests.RequestException as e:
        logging.error(f"요청 실패: {e}")
        return None
    except Exception as e:
        logging.error(f"파싱 실패: {e}")
        return None
    
    max_tag_count = 0
    news_content_div = None

    # 하위 <div> 요소 개수 제한을 2에서 10까지 시도
    for div_limit in range(2, 10):
        for div in soup.find_all('div'):
            # 숨겨진 콘텐츠로 파싱 되는지 확인
            if div.get('style') and 'display:none' in div.get('style'):
                logging.error(f"콘텐츠 숨김으로 인한 파싱 불가 {url}")
            
            num_nested_divs = len(div.find_all('div'))
            if num_nested_divs >= div_limit:
                continue
            
            # 각 태그의 개수를 합산
            tag_count = sum(len(div.find_all(tag)) for tag in tags_to_check)
            
            # 태그 개수가 최대인 <div>를 찾음
            if tag_count > max_tag_count and len(div.get_text(strip=True)) >= 300:
                max_tag_count = tag_count
                news_content_div = div
        
        # 가장 많은 태그를 가진 <div>의 텍스트가 300자 이상인지 확인
        if news_content_div:
            content = news_content_div.get_text(strip=True)
            if len(content) >= 300:
                return content
    
    return None


# 예시 URL을 사용하여 함수 호출
url = news_df["Original Link"][0]
print(extract_news_content(url))


/TV조선 방송화면 캡처국민연금이 가상화폐에 대한 간접 투자를 확대하고 있다.코인 전문매체 코인데스크는 16일(현지시간) 최근 공개된 증권거래위원회(SEC) 자료를 인용해 국민연금이 올해 2분기에 마이크로스트레티지 주식 약 3400만 달러(460억 원) 상당을 매입했다고 보도했다.마이크로스트래티지는 가장 많은 비트코인을 보유한 회사다.약 2천만 개에 달하는 비트코인 전체 공급량의 1% 이상인 22만6500개를 보유 중이다.국민연금은 마이크로스트래티지가 이달 초 1대 10의 주식 분할 전에 평균 1377.48달러에 2만4500주를 매입했다.정확하게 3374만8260달러(약 457억 원) 규모다.주식 분할 후 국민연금이 보유한 주식 수는 24만5천 주로 늘었다.이날 종가(133.04달러) 기준 보유 금액은 3259만4800달러(약 441억 원)로 3.4% 손실을 기록 중이다.코인데스크는 "마이크로스트래티지가 코인 최대 기업 보유자인 만큼 국민연금의 주식 매입은 비트코인(BTC)에 대한 간접 투자로 볼 수 있다"고 전했다.국민연금이 가상화폐 관련 기업 주식을 매입한 것은 처음은 아니다.코인데스크에 따르면 국민연금은 미 가상화폐 거래소 코인베이스 주식 22만9807주도 보유하고 있다.국민연금은 지난해 코인베이스 주식을 평균 70.5달러에 28만2673주를 매입했다.


In [133]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(news_df['Original Link'][0])
docs = loader.load()
print(docs)
text = docs[0]
text

[Document(metadata={'source': 'https://biz.chosun.com/international/international_economy/2024/08/17/JDIT2LDAJBBTTPHPED53JUYGEE/?utm_source=naver&utm_medium=original&utm_campaign=biz', 'title': '국민연금, 가상화폐 간접 투자… 美 비트코인 최다 보유 기업 주식 매입 - 조선비즈', 'description': '국민연금, 가상화폐 간접 투자 美 비트코인 최다 보유 기업 주식 매입  국민연금이 가상화폐에 대한 간접 투자를 확대하고 있다. 코인 전문 매체 코인데스크는 16일현지 시각 최근 공개된 증권거래위원회SEC 증권 신고서를 인용해 국민연금이 올해 2분기 약 3400만달러46', 'language': 'ko'}, page_content='국민연금, 가상화폐 간접 투자… 美 비트코인 최다 보유 기업 주식 매입 - 조선비즈\n\n\n')]


Document(metadata={'source': 'https://biz.chosun.com/international/international_economy/2024/08/17/JDIT2LDAJBBTTPHPED53JUYGEE/?utm_source=naver&utm_medium=original&utm_campaign=biz', 'title': '국민연금, 가상화폐 간접 투자… 美 비트코인 최다 보유 기업 주식 매입 - 조선비즈', 'description': '국민연금, 가상화폐 간접 투자 美 비트코인 최다 보유 기업 주식 매입  국민연금이 가상화폐에 대한 간접 투자를 확대하고 있다. 코인 전문 매체 코인데스크는 16일현지 시각 최근 공개된 증권거래위원회SEC 증권 신고서를 인용해 국민연금이 올해 2분기 약 3400만달러46', 'language': 'ko'}, page_content='국민연금, 가상화폐 간접 투자… 美 비트코인 최다 보유 기업 주식 매입 - 조선비즈\n\n\n')

In [104]:
from bs4 import BeautifulSoup
import requests
import certifi

def extract_news_content(url):
    # 체크할 태그 목록 설정 : 해당 태그가 많은 <div>가 뉴스콘텐츠가 있는 div
    tags_to_check = ['br', 'p']
    
    # 브라우저 User-Agent 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }
    
    # URL에서 HTML 콘텐츠 가져오기
    response = requests.get(url, headers=headers, verify=certifi.where())
    response.encoding = 'utf-8'  # 적절한 인코딩으로 설정
    soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
    
    max_tag_count = 0
    news_content_div = None

    # 하위 div 요소 개수 제한을 2에서 시작하여 5까지 시도
    for div_limit in range(2, 10):
        for div in soup.find_all('div'):
            print(f"\n div_limit : {div_limit} 하위 div 요소 개수 : {len(div.find_all('div'))}")
            print(div)
            tag_count = 0
            
            # 각 태그의 개수를 합산
            for tag in tags_to_check:
                tag_count += len(div.find_all(tag))
            
            # 하위 div 요소 개수를 제한하고, 태그 개수가 최대인 div를 찾음
            if len(div.find_all('div')) < div_limit and tag_count > max_tag_count:
                max_tag_count = tag_count
                news_content_div = div
        
        # 가장 많은 지정된 태그를 가진 <div>의 텍스트가 300자 이상인지 확인
        if news_content_div and len(news_content_div.get_text(strip=True)) >= 300:
            return news_content_div.get_text(strip=True)
    
    # 만약 조건을 만족하는 div를 찾지 못했다면 None 반환
    return None

# Example usage
url = news_df["Original Link"][0]
print(extract_news_content(url))



 div_limit : 2 하위 div 요소 개수 : 1
<div id="fusion-app"><div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div></div>

 div_limit : 2 하위 div 요소 개수 : 0
<div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div>

 div_limit : 3 하위 div 요소 개수 : 1
<div id="fusion-app"><div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div></div>

 div_limit : 3 하위 div 요소 개수 : 0
<div data-fusion-collection="layouts" data-fusion-message="Could not render component [layouts:article]" data-fusion-type="article" id="article" style="display:none"></div>

 div_limit : 4 하위 div 요소 개수 : 1
<div id="fusion-app"><div data-fusion-collection="layouts" data-fusion-message="Could not

In [95]:
from bs4 import BeautifulSoup
import requests
import certifi

def extract_news_content(url):
    # 체크할 태그 목록 설정 : 해당 태그가 많은 <div> 또는 <section> 등이 뉴스 콘텐츠가 있을 가능성이 큼
    tags_to_check = ['br', 'p']
    container_tags = ['div', 'section', 'article']  # 콘텐츠를 포함할 수 있는 태그들
    
    # 브라우저 User-Agent 설정
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }
    
    # URL에서 HTML 콘텐츠 가져오기
    response = requests.get(url, headers=headers, verify=certifi.where())
    response.encoding = 'utf-8'  # 적절한 인코딩으로 설정
    soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
    
    max_tag_count = 0
    news_content_div = None
    
    # 하위 container_tag 요소 개수 제한을 2에서 10까지 시도
    for div_limit in range(2, 11):
        print(div_limit)
        for container_tag in container_tags:
            print(container_tag)
            for container in soup.find_all(container_tag):
                print("section: ",soup.find_all('section'))
                print("articles: ",soup.find_all('article'))
                tag_count = 0
                
                # 각 태그의 개수를 합산
                for tag in tags_to_check:
                    tag_count += len(container.find_all(tag))
                
                # 텍스트 길이를 기준으로 필터링
                text_length = len(container.get_text(strip=True))
                
                # 하위 요소 개수 제한과 텍스트 길이를 조건으로 사용
                if len(container.find_all(container_tag)) < div_limit and tag_count > max_tag_count and text_length >= 300:
                    max_tag_count = tag_count
                    news_content_div = container
            
        # 텍스트가 300자 이상인 콘텐츠를 찾은 경우 반환
        if news_content_div:
            return news_content_div.get_text(strip=True)
        print()
    # 조건을 만족하는 콘텐츠가 없을 경우 None 반환
    return None

# Example usage
url = news_df["Original Link"][0]
print(extract_news_content(url))


2
div
section:  []
articles:  []
section:  []
articles:  []
section
article

3
div
section:  []
articles:  []
section:  []
articles:  []
section
article

4
div
section:  []
articles:  []
section:  []
articles:  []
section
article

5
div
section:  []
articles:  []
section:  []
articles:  []
section
article

6
div
section:  []
articles:  []
section:  []
articles:  []
section
article

7
div
section:  []
articles:  []
section:  []
articles:  []
section
article

8
div
section:  []
articles:  []
section:  []
articles:  []
section
article

9
div
section:  []
articles:  []
section:  []
articles:  []
section
article

10
div
section:  []
articles:  []
section:  []
articles:  []
section
article

None


In [49]:
import pandas as pd
import logging

# 로그 설정: INFO 레벨의 로그를 콘솔에 출력
logging.basicConfig(level=logging.INFO, format='%(message)s')

# CSV 파일에서 데이터프레임으로 읽어오기
news_df = pd.read_csv('news_data.csv')

# 'content'라는 새로운 컬럼 생성
news_df['content'] = ""

# 링크에서 원문 모두 스크래핑하여 'content' 컬럼에 추가
for i in range(len(news_df)):
    url = news_df["Original Link"][i]
    try:
        news_df.at[i, 'content'] = extract_news_content(url)
    except requests.exceptions.RequestException as e:
        # 인증이 거부된 경우 로그 출력
        logging.info(f"{i} {url} 에서 인증을 거부했습니다. 에러 메시지: {str(e)}")

# 결과 확인
print(news_df.head())


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
135 https://www.newsen.com/news_view.php?uid=202408130752322410 에서 인증을 거부했습니다. 에러 메시지: HTTPSConnectionPool(host='www.newsen.com', port=443): Max retries exceeded with url: /news_view.php?uid=202408130752322410 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
378 https://vop.co.kr/A00001659511.html 에서 인증을 거부했습니다. 에러 메시지: HTTPSConnectionPool(host='vop.co.kr', port=443): Max retries exceeded with url: /A00001659511.html (Caused by SSLError(SSLError(1, '[SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:1007)')))
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
409 http://www.sentv.co.kr/news/vi

  Category                                Title  \
0       주식  [경제PICK] 주식 시세처럼...실시간 '부동산 통계 시스템'   
1       주식    [단독] 도이치 주식 최다 수익자, 윤 대통령 고액 후원했다   
2       주식  애물단지 전락한 물납주식…상속인 되살 때 최대 50% 할인해준다   
3       주식     30대그룹 공익법인, 계열사 보유주식 늘리고 기부액 줄였다   
4       주식   정부, 안 팔리는 '물납주식', 상속인 다시 사도록 규제 푼다   

                                       Original Link  \
0  https://www.ytn.co.kr/_ln/0102_202408131715390968   
1  https://www.ohmynews.com/NWS_Web/View/at_pg.as...   
2     https://www.hankyung.com/article/202408142426i   
3  https://www.newsis.com/view/NISX20240812_00028...   
4  https://www.nocutnews.co.kr/news/6195658?utm_s...   

                  Publication Date  \
0  Tue, 13 Aug 2024 17:15:00 +0900   
1  Wed, 14 Aug 2024 07:03:00 +0900   
2  Wed, 14 Aug 2024 11:03:00 +0900   
3  Tue, 13 Aug 2024 06:00:00 +0900   
4  Wed, 14 Aug 2024 10:31:00 +0900   

                                             content  
0  [앵커]주식 시세처럼, 실시간 부동산 통계 시스템! 시시각각 변하는 주가처럼 부동산...  
1  큰사진보기▲도이치모터스ⓒ 

In [50]:

# 데이터프레임을 CSV 파일로 저장
news_df.to_csv('news_data.csv', index=False)
print("CSV 파일이 'news_data.csv'로 저장되었습니다.")

CSV 파일이 'news_data.csv'로 저장되었습니다.


In [59]:
# 글자 수가 300 이하인 content를 찾기
short_contents = news_df[news_df['content'].str.len() <= 300]

# 해당 행의 인덱스와 본문을 출력
for index, row in short_contents.iterrows():
    print(f"Index: {index}, Content: {row['content']}")
    print(news_df["Original Link"][index])
    print()

Index: 3, Content: 오늘의 헤드라인" 대통령 경축사에 엇갈린 여야"새로운 비전" vs "日 언급 못해"여야는 15일 윤석열 대통령의 광복절 경축사에 대해 엇갈린 평가를 냈다. 국민의힘은 윤 대통령이 통일을 위한 새로운 비전을 제시했다고 호평한 반면 민주당 등 야당은 일제에 관한 과거사를 언급하지 않았다며 최악의 경축사라고 혹평했다.

한지아 국민의힘 수석대변인은 논평을 내고 "윤 대통령께서는 오늘 '더 큰 자유와 기회를 누리는' 통일 대한민국을 향한
https://www.newsis.com/view/NISX20240812_0002847314

Index: 4, Content: 00
https://www.nocutnews.co.kr/news/6195658?utm_source=naver&utm_medium=article&utm_campaign=20240814103039

Index: 5, Content: 이 뉴스 공유하기닫기카카오톡페이스북X카카오스토리페이스북메신저네이버밴드네이버블로그URL복사
https://www.yna.co.kr/view/AKR20240814046900009?input=1195m

Index: 6, Content: 최상목 부총리, 15일 국유재산정책심의위 주재"국유재산 유지·보존→개발·활용, 국민 활용 적극 추진"노후청사·국유지 청년주택, 시니어 레지던스 등 공급물납주식 감액규정 신설, 투자형은 증권사가 주관토록등록 2024-08-14 오전 10:30:00수정 2024-08-14 오전 10:30:00가가
http://www.edaily.co.kr/news/newspath.asp?newsid=02063126638987320

Index: 7, Content: 회원님은부터“asdf”를 구독하고 계십니다.아래 ‘구독취소’ 버튼을 클릭해서 구독을 ‘취소’하실 수 있습니다.해당 구독 취소의 효과는 “”에 한정되며, 서울경제 뉴스레터 수신에 대한 설정값이나 다른 뉴스레터 수신여부에는 영향을 끼치지 않습니다.
https://www.sedaily